<a href="https://colab.research.google.com/github/snow-white2024/AIFFEL_quest_cr/blob/master/10_%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8%EB%A1%9C_%EB%A7%8C%EB%93%9C%EB%8A%94_%EB%8C%80%ED%99%94%ED%98%95_%EC%B1%97%EB%B4%87_%5B%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1. 데이터 수집하기

In [2]:
import os
import urllib.request
import pandas as pd

# 데이터 다운로드 (GitHub에서 직접 가져오기)
dataset_url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
dataset_path = "ChatbotData.csv"

# 파일 다운로드
urllib.request.urlretrieve(dataset_url, dataset_path)

# 데이터 불러오기
df = pd.read_csv(dataset_path)

# 데이터 크기 및 샘플 확인
print(f"데이터 크기: {df.shape}")
print(df.head())


데이터 크기: (11823, 3)
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


# Step 2. 데이터 전처리하기

In [3]:
import re

# 전처리 함수 정의
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 소문자 변환 및 공백 제거
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)  # 구두점과 단어 분리
    sentence = re.sub(r"[^ㄱ-ㅎ가-힣0-9a-zA-Z?.!,]+", " ", sentence)  # 특수문자 제거
    sentence = re.sub(r"\s+", " ", sentence)  # 다중 공백 제거
    return sentence

# 데이터 전처리 적용
df['Q'] = df['Q'].apply(preprocess_sentence)
df['A'] = df['A'].apply(preprocess_sentence)

# 전처리된 데이터 확인
print(df.head())


                 Q              A  label
0         12시 땡 !    하루가 또 가네요 .       0
1      1지망 학교 떨어졌어    위로해 드립니다 .       0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠 .       0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠 .       0
4          ppl 심하네   눈살이 찌푸려지죠 .       0


# Step 3. SubwordTextEncoder 사용하기

In [4]:
import tensorflow as tf  # TensorFlow 라이브러리 임포트
import tensorflow_datasets as tfds

# 질문과 답변 데이터 리스트
questions = df['Q'].tolist()
answers = df['A'].tolist()

# 토크나이저 훈련 (SubwordTextEncoder 사용)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

# 시작/종료 토큰 정의
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2  # 시작/종료 토큰 포함

# 정수 인코딩 및 패딩
MAX_LENGTH = 40  # 최대 문장 길이 설정

def tokenize_and_filter(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        tokens = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
        if len(tokens) <= MAX_LENGTH:
            tokenized_sentences.append(tokens)
    return tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_sentences, maxlen=MAX_LENGTH, padding='post')

# 질문 및 답변 토큰화 및 패딩
questions = tokenize_and_filter(questions)
answers = tokenize_and_filter(answers)

print(f"단어장 크기: {VOCAB_SIZE}")
print(f"토큰화된 질문 샘플: {questions[0]}")


단어장 크기: 8168
토큰화된 질문 샘플: [8166 7905 4199 3051   38 8167    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


# Step 4. 트랜스포머 모델 구성하기

In [5]:
import tensorflow as tf

# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    d_k = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(d_k)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output

# 멀티 헤드 어텐션
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask):
        batch_size = tf.shape(q)[0]
        q = self.split_heads(self.query_dense(q), batch_size)
        k = self.split_heads(self.key_dense(k), batch_size)
        v = self.split_heads(self.value_dense(v), batch_size)

        attention_output = scaled_dot_product_attention(q, k, v, mask)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention_output, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)
        return output

# 트랜스포머 모델 구성
def build_transformer(vocab_size, num_layers=2, d_model=128, num_heads=8, dff=512):
    inputs = tf.keras.layers.Input(shape=(MAX_LENGTH,))
    embedding = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    x = embedding

    for _ in range(num_layers):
        x = MultiHeadAttention(d_model, num_heads)(x, x, x, None)

    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# 모델 생성 및 컴파일
transformer = build_transformer(VOCAB_SIZE)
transformer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

transformer.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 40)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 40, 128)        │      1,045,504 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, None, 128)      │         66,048 │ embedding[0][0],       │
│ (MultiHeadAttention)      │                        │                │ embedding[0][0],       │
│                           │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, None, 128)      │         66,048 │ multi_head_attention[… │
│ (MultiHeadAttention)      │                        │                │ multi_head_attention[… │
│                           │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, None, 8168)     │      1,053,672 │ multi_head_attention_… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,231,272 (8.51 MB)

 Trainable params: 2,231,272 (8.51 MB)

 Non-trainable params: 0 (0.00 B)

# Step 4-1. 모델 학습하기

In [6]:
# Step 1. 데이터셋 준비
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# TensorFlow Dataset으로 변환
dataset = tf.data.Dataset.from_tensor_slices((questions, answers))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Step 2. 콜백 함수 정의 (모델 체크포인트 저장)
checkpoint_path = "transformer_chatbot.weights.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    monitor="loss",
    verbose=1
)

# Step 3. 모델 학습 (EPOCHS 지정)
EPOCHS = 50  # 원하는 만큼 조정 가능

transformer.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

print("✅ 모델 학습 완료!")


Epoch 1/50
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7807 - loss: 3.0142
Epoch 1: loss improved from inf to 2.04110, saving model to transformer_chatbot.weights.h5
185/185 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.7808 - loss: 3.0090
Epoch 2/50
182/185 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8059 - loss: 1.6499
Epoch 2: loss improved from 2.04110 to 1.66481, saving model to transformer_chatbot.weights.h5
185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8058 - loss: 1.6502
Epoch 3/50
182/185 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8093 - loss: 1.5028
Epoch 3: loss improved from 1.66481 to 1.39016, saving model to transformer_chatbot.weights.h5
185/185 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8095 - loss: 1.5004
Epoch 4/50
184/185 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8338 - loss: 1.2558
Epoch 4: loss improved from 1.39016 to 1.25287, saving model to transformer_chatbot.weights.h5
185/185 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - acc

# Step 5. 모델 평가하기

In [ ]:
import numpy as np

def predict(sentence):
    sentence = preprocess_sentence(sentence)
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    sentence = tf.keras.preprocessing.sequence.pad_sequences([sentence], maxlen=MAX_LENGTH, padding='post')

    prediction = transformer.predict(sentence)
    predicted_id = np.argmax(prediction[0], axis=-1)

    result = tokenizer.decode([i for i in predicted_id if i < tokenizer.vocab_size])
    return result

# 테스트
print("챗봇 테스트:")
while True:
    q = input("User: ")
    if q.lower() == "exit":
        break
    print("Bot:", predict(q))


챗봇 테스트:
User: 오늘 날씨가 추워.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 731ms/step
Bot: 크게 마음도 나가바랍니다나가
User: 좋은 아침이야.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Bot: 맛나게 좀 할게요할게요할게요
User: 너 말투가 왜 그래?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Bot: 저도 나 권권하지  . 
User: 너 지금 장난하냐
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Bot:  . 재미있나봐요 . 
User: ?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Bot: 맞기만 
